



1. 数据爬取的主要目的就是从非结构性的数据源来提取到结构性的数据。比如说网页，它是一个非结构性数据，我们把很多网站上的网页获取下来之后，把它们结构化成我们自己定义的结构。
2. 但是当我们提取了这些数据之后如何将数据返回：最简单的是把返回的数据放入一个字典当中，通过字典返回给Scrapy
3. 虽然字典(dict)很好用，但是dict缺乏一些结构性的元素，比如容易打错字段的名字，在统一处理字典时容易出错 
```
爬虫A返回字典
A:
{
    'fav_nums':
    'comment_nums:'
}
爬虫B返回字典
B:
{
    'fav_num':
    'comment_num'
}
```
4. 为了进行完整的格式化，Scrapy提供了Item类(items.py)，Item类允许用户自定义字段，比如自定义'Article'Item类，在Article统一定义各种爬虫所共有的字段
5. 当我们队Item进行实例化之后，在对应的py文件里进行yield操作时，比如通过parse()获得了一个Item类，然后直接把Item类yield就行了，Scrapy在接受yield时，发现这是一个Item实例的时候，会直接将该Item路由到'piplines.py'文件里
    * 好处：在'piplines.py'文件里集中处理数据的保存,去重...
```
def parse(self, response):  
        post_urls = response.css("#archive .floated-thumb .post-thumb a::attr(href)").extract()
        for post_url in post_urls:
            yield Request(url=parse.urljoin(response.url, post_url), callback=self.parse_detail)
```

#### *拓展延伸
详情页：<img src="1.png" width='400'>
列表页：<img src="2.png" width='400'>
* 因为对于Item类的处理是在parse_detail()方法中定义并且进行的，如果我们想在Item中保存对应文章列表页的图片，需要将列表页的图片传递到parse_detail()进行处理
* 通过Request()方法中的meta参数进行传参，meta参数是一个dict(因为我们不仅可以只传递图片，还可以传递任意我们想要的参数，这些参数通过dict进行保存)
    ```
    post_nodes = response.css("#archive .floated-thumb .post-thumb a")
        for post_node in post_nodes:
            image_url = post_node.css("img::attr(src)").extract_first("")
            post_url = post_node.css("::attr(href)").extract_first("")
            yield Request(url=parse.urljoin(response.url, post_url), meta={"front_image_url": image_url}
                ,callback=self.parse_detail)
    ```
<img src='3.png'>
* 然后再parse_details()用变量接受meta参数
```
def parse_detail(self, response):
        # response.meta.get("front_image_url", "")是一种不会抛异常的方法，本质和response.meta['front_image_url']相同
        front_image_url = response.meta.get("front_image_url", "")
```

## item.py

1. 区别于其他数据类型(int, string...), 在Item类中只有一个字段类型：scrapy.Field()
```
class JobBoleArticleIte(scrapy.Item):
        # 标题
        title = scrapy.Field()
        # 创建时间
        create_date = scrapy.Field()
        # 当前页面的url
        url = scrapy.Field()
        # URL对应的MD5
        url_object_id = scrapy.Field()
        # 封面图
        front_image_url = scrapy.Field()
        # 封面图本地存放路径
        front_image_path = scrapy.Field()
        # 点赞数
        praise_nums = scrapy.Field()
        # 评论数
        comment_nums = scrapy.Field()
        # 收藏数
        fav_nums = scrapy.Field()
        # 标签
        tags = scrapy.Field()
        # 内容
        content = scrapy.Field()
        pass
```
2. 填充Item类中对应的字段
    * 首先需要在py文件中导入Item类
        * from ArticleSpider.items import JobBoleArticleItem
    * 实例化Item类
        * article_item = JobBoleArticleItem()
    * 填充Item对象
        * article_item["create_date"] = create_date......
    * 调用yield() # Scrapy在接受yield时，发现这是一个Item实例的时候，会直接将该Item路由到'piplines.py'文件里
        * yield article_item
    * 使Pipline生效，从而能够处理Item
        ```
        'settings.py'
        # Configure item pipelines
        # See https://doc.scrapy.org/en/latest/topics/item-pipeline.html
        ITEM_PIPELINES = {
            'ArticleSpider.pipelines.ArticlespiderPipeline': 300,
        }
        ```
<img src='4.png'>
3. 给Item类中的字段添加功能(保存到本地......)
    * scrapy提供了一种自动下载图片的机制，只需要在'settings.py'进行配置
        * 管道(PIPELINES)对象:数字表示数据在管道中流经的顺序,即处理类对数据处理的顺序：小->大,先->后
        ```
        ITEM_PIPELINES = {
                'ArticleSpider.pipelines.ArticlespiderPipeline': 300,
                'scrapy.piplines.images.ImagesPipeline': 1
        }
        ```
        * 配置Item类中哪个字段代表的是图片，然后PIPLINE到Item中去取对应图片的字段
        ```
        ITEM_PIPELINES = {
                'ArticleSpider.pipelines.ArticlespiderPipeline': 300,
                'scrapy.piplines.images.ImagesPipeline': 1
                }
        IMAGE_URLS_FIELD = 'front_image_url'
        ```
        * 配置图片下载路径
            * 这里有必要对路径的表示进行说明 
                * os.path.abspath(os.path.dirname(\__file\__))   
                ->G:\Python\PythonNote\Scrapy\ArticleSpider
                * os.path.dirname(\__file\__))  
                ->G:/Python/PythonNote/Scrapy/ArticleSpider
                * os.path.dirname(os.path.abspath(\__file\__))  
                ->G:\Python\PythonNote\Scrapy\ArticleSpider
            * os.path.dirname()只是获取目录名字,而不是目录路径,在windows下反斜杠'\'才表示路径
            * os.path.abspath()才是获取文件目录的地址
            ```
            ITEM_PIPELINES = {
                'ArticleSpider.pipelines.ArticlespiderPipeline': 300,
                'scrapy.piplines.images.ImagesPipeline': 1
            }
            IMAGES_URLS_FIELD = 'front_image_url'

            import os

            project_dir = os.path.abspath(os.path.dirname(__file__))
            IMAGES_STORE = os.path.join(project_dir, 'images')
            ```
        * 下载处理图片的相关库(PIL)
            * pip install -i https://pypi.douban.com/simple pillow
        * 处理异常：raise ValueError('Missing scheme in request url: %s' % self._url)
            * 在PIPLINE处理图片时,IMAGE_URLS_FIELD被PIPLINE默认为数组类型，所以需要在相关py文件下把front_image_url设置为数组
            * article_item["front_image_url"] = [front_image_url]
<img src='5.png'>
    * 图片既然已经保存下来了，就需要把图片的路径提取出来，然后把图片路径和front_image_path字段绑定在一起，以便之后进行存入数据库操作
    * 对于图片下载有与之对应的'scrapy.piplines.images.ImagesPipeline'类可以进行操作，但是把图片路径和front_image_path绑定在一起的处理类并没有现成的，
    所以需要自定义一个能够处理对应功能的PIPLINE类
    #### (拓展)自定义PIPLINE类的编写方式：
         * 注意重载
         * 注意返回item,因为处于管道后面的管道类同样需要处理item，如果不返回item，则处于序列后面的PIPELINE类就没有item可以处理了
    * 自定义的PIPELINE类需要继承ImagesPipeline里的部分功能重载，在此基础上自定义其他功能
        ```
            # 观察这个方法体的循环，就知道为什么我们需要把front_image_url设为数组形式的数据了
            # 获得数组元素后，通过Request传递给scrapy进行下载
            def get_media_requests(self, item, info):
                return [Request(x) for x in item.get(self.images_urls_field, [])]
        ```
        ```
            # 重载这个方法之后就可以获得到文件的下载路径，路径存放在results参数里
            def item_completed(self, results, item, info):
                if isinstance(item, dict) or self.images_result_field in item.fields:
                    item[self.images_result_field] = [x for ok, x in results if ok]
                return item
        ```
        <img src='6.png'>
        ```
        from scrapy.pipelines.images import ImagesPipeline
        
        class ArticleImagePipLine(ImagesPipeline):
            def item_completed(self, results, item, info):
                # 通过results获得文件实际的存储路径
                for ok, value in results:
                    image_file_path = value['path']
                item['front_image_path'] = image_file_path

                return item
        ```
        * results[1].path:文件存放路径
        * results[1].url:文件url
        * results[0]:下载状态(是否成功)  
    * 下一个PIPLINE接受数据(成功接收到front_image_path)
    <img src='7.png'>
    * 获取到数据之后就可以进行保存数据库...操作了
4. 把不定长的url地址转换成定长的md5编码
    * 新建相应的方法(get_md5)
    ```
    import hashlib
    def get_md5(url):
        if isinstance(url, str):
            url = url.encode('utf-8')
        m = hashlib.md5()
        m.update(url)
        return m.hexdigest()
    ```